In [3]:
import lightgbm as lgb
import numpy as np
import sklearn.metrics

In [53]:
base = 'gameevents_0[1-9][0-9].csv'
train_X = np.load(f'{base}_game_states.npy')
train_y = np.load(f'{base}_labels.npy')
train_data = lgb.Dataset(train_X, train_y)

In [61]:
param = {'num_leaves': 1000, 'objective': 'binary', 'metric': 'binary_logloss', 'boosting': 'dart'}
bst = lgb.train(param, train_data, 200)

[LightGBM] [Info] Number of positive: 440788, number of negative: 447567
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057770 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 443
[LightGBM] [Info] Number of data points in the train set: 888355, number of used features: 71
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.496185 -> initscore=-0.015262
[LightGBM] [Info] Start training from score -0.015262


In [41]:
test_X = np.load('gameevents_000.csv_game_states.npy')
test_y = np.load('gameevents_000.csv_labels.npy')

In [62]:
print(sklearn.metrics.classification_report(test_y, bst.predict(test_X) > .5))

              precision    recall  f1-score   support

           0       0.68      0.70      0.69     50919
           1       0.70      0.68      0.69     52701

    accuracy                           0.69    103620
   macro avg       0.69      0.69      0.69    103620
weighted avg       0.69      0.69      0.69    103620

